# SPARQL Endpoint for Graph

(C) 2024 by [Damir Cavar](http://damir.cavar.me/)

In this notebook we try to communicate with DBpedia. You can test your query here:

- [DBpedia SPARQL endpoint]()

Below is an example of a communication with our own SPARQL endpoint.


In [ ]:
import urllib, json
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

In [ ]:
sparql.setQuery("""PREFIX schema: <http://schema.org/>
PREFIX yago: <http://yago-knowledge.org/resource/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT * WHERE {
  ?subj  ?pred "Elvis Presley" .
} 
LIMIT 10""")

In [ ]:
sparql.setQuery("""PREFIX dbres: <http://dbpedia.org/resource/>

DESCRIBE dbres:United_States
LIMIT 10""")

In [ ]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

## Query on Our Own Server

If you set up your own [Virtuoso server](https://virtuoso.openlinksw.com/), you can store ontologies and knowledge graphs in it and query them as described below.

The server is located at `http://localhoist:8890/sparql`. Our basic graph has the URI `http://localhoist:8890/DAV/cis`.

We can query the graph using SPARQL and just urllib:

In [ ]:
def sparqlQuery(query, baseURL, format="application/json"):
	params={
		"default-graph": "http://localhoist:8890/DAV/cis",
		"named_graphs": [ "http://localhoist:8890/DAV/cis" ],
		"should-sponge": "soft",
		"query": query,
		"debug": "on",
		"timeout": "",
		"format": format,
		"save": "display",
		"fname": ""
	}
	querypart=urllib.parse.urlencode(params)
	querypart = querypart.encode('utf-8')
	response = urllib.request.urlopen(baseURL, querypart)
	data = response.read()
	return json.loads(data.decode('utf-8'))

The query function requires a query text and the endpoint URI. Optionally one can specify the returned data format. By default it is JSON.

In [ ]:
dsn="http://localhost:8890/sparql"
query="""SELECT ?s ?label WHERE {
   ?s rdfs:subClassOf <http://www.esslli.org/kg/sec/Corporation> ;
      rdfs:label      ?label
} LIMIT 30""" # % dsn 

data = sparqlQuery(query, dsn)
print("Retrieved data:\n" + json.dumps(data, sort_keys=True, indent=4))

This can be simplified with the SPARQLWrapper as in the following:

In [ ]:
sparql = SPARQLWrapper(dsn)
sparql.setReturnFormat(JSON)
sparql.setQuery(query)
result = sparql.query().convert()

In [ ]:
print(json.dumps(result, sort_keys=True, indent=4))